In [0]:
%sql
/*
NoteBook que simula el Job JP_PATRIF_GAR_0500_ARC_ACEP_LIN3 Que Genera el Sumario de archivos de Aceptacion del 
SUBPROCESO 354
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("GAR")))

In [0]:
statement_001 = query.get_statement(
"GAR_0500_ARC_ACEP_LIN3_SUMARIO.sql",
DELTA_SUMARIO_SOLMARC = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.delta_0399_arc_acep_lin2_tf_200_campos_{params.sr_folio}"
)

In [0]:
df = spark.sql(statement_001) 
if conf.debug:
    display(df)
    display(str(df.count()) + " Registros Totales")

In [0]:
temp_delta = 'DELTA_0500_ARC_ACEP_LIN3_SUMARIO' + '_' + params.sr_folio
db.write_delta(temp_delta,spark.sql(statement_001) , "overwrite")
if conf.debug:
    display(db.read_delta(temp_delta))

In [0]:
delta_df = spark.table(f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.DELTA_0500_ARC_ACEP_LIN3_SUMARIO_"+ params.sr_folio)
display(delta_df)

In [0]:
statement_002 = query.get_statement(
"GAR_0600_ARCHIVO_SOLMARCA.sql",
DELTA_SOLMARC_ENCABEZADO = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.delta_0300_arc_acep_lin1_encabezado_{params.sr_folio}",
DELTA_SOLMARC_DETALLE = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.delta_0400_arc_acep_lin2_detalle_{params.sr_folio}",
DELTA_SOLMARC_SUMARIO = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.delta_0500_arc_acep_Lin3_sumario_{params.sr_folio}"
)

In [0]:
df_final = spark.sql(statement_002) 
if conf.debug:
    display(df_final)
    display(str(df_final.count()) + " Registros Totales")

In [0]:
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual = datetime.now(mexico_tz).strftime("%Y%m%d")

# Crea el nombre completo del archivo concatenando la ubicación externa, el camino del repositorio de errores y la fecha actual
full_file_name = (
        conf.external_location
        + conf.err_repo_path
        + conf.output_file_name_bis43_acept_1
        + fecha_actual
        + conf.output_file_name_bis43_acept_2
        + ".txt"
    )

if conf.debug:
    display(full_file_name)


In [0]:
# Genera el archivo y decide si calcular MD5 o no

file_manager.generar_archivo_flexible(
    df=df_final,
    full_file_name=full_file_name,
    opciones={ # Aqui pones las opciones que necesites.
        "header": "false",
        "encoding": "ISO-8859-1",
        #"delimiter": ",",
        "quote": '"',
        "charset": "latin1",
        "escapeQuotes": False,
        "ignoreLeadingWhiteSpace": "false",
        "ignoreTrailingWhiteSpace": "false"
    },
    calcular_md5=False
)


In [0]:
# Inicializa la clase
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Llama al método para generar el archivo
full_file_name = full_file_name

# Genera el archivo y decide si calcular MD5 o no
file_manager.generar_archivo_ctindi(
    df_final=df_final,  # DataFrame que se va a guardar
    full_file_name=full_file_name,
    header=conf.header,
    calcular_md5=False  # Cambia a False si no quieres calcular el MD5
)

In [0]:
#En caso de que se requiera validar el archivo en la maquina local
#dbutils.fs.cp('abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/PTCFTA_20250801_001_ACEPMARCA43.txt', 
#'/Volumes/dbx_mit_dev_1udbvf_workspace/default/doimss_cararch/PTCFTA_20250801_001_ACEPMARCA43.txt')